In [0]:
%pip install gql
%pip install requests_toolbelt
%restart_python

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType, DoubleType, BooleanType, MapType
import requests
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

In [0]:
client_id = dbutils.secrets.get(scope="warcraftlogs", key="client_id")
client_secret = dbutils.secrets.get(scope="warcraftlogs", key="client_secret")

token_url = "https://www.warcraftlogs.com/oauth/token"
response = requests.post(token_url, data={
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret
})

access_token = response.json()["access_token"]
print("Access token:", access_token)


In [0]:
def _get_access_token():
    client_id = dbutils.secrets.get(scope="warcraftlogs", key="client_id")
    client_secret = dbutils.secrets.get(scope="warcraftlogs", key="client_secret")

    response = requests.post(
        "https://www.warcraftlogs.com/oauth/token",
        data={"grant_type": "client_credentials",
              "client_id": client_id,
              "client_secret": client_secret}
    )
    return response.json()["access_token"]

def get_event_data_spark(report_code: str, fight_start: int, fight_end: int, data_type: str = "DAMAGE"):
    access_token = _get_access_token()
    
    transport = RequestsHTTPTransport(
        url="https://www.warcraftlogs.com/api/v2/client",
        headers={"Authorization": f"Bearer {access_token}"},
        use_json=True,
    )
    client = Client(transport=transport, fetch_schema_from_transport=False)

    all_events = []
    current_start = fight_start

    while current_start < fight_end:
        query = gql(f"""
        query {{
          reportData {{
            report(code: "{report_code}") {{
              events(startTime: {current_start}, endTime: {fight_end}, dataType: {data_type}) {{
                data
                nextPageTimestamp
              }}
            }}
          }}
        }}
        """)

        result = client.execute(query)
        event_chunk = result['reportData']['report']['events']['data']
        all_events.extend(event_chunk)

        next_page = result['reportData']['report']['events'].get('nextPageTimestamp')
        if not next_page:
            break
        current_start = next_page

    # Create Spark DataFrame directly from list of dicts
    spark = SparkSession.builder.getOrCreate()
    return spark.createDataFrame(all_events)

In [0]:
df = get_event_data_spark(
    report_code="rVnDB4fcb9qZAPLv",
    fight_start=8021594,
    fight_end=8415041,
    data_type="DamageDone"  # or "CASTS", "DEATHS", etc.
)

display(df)

In [0]:
transport = RequestsHTTPTransport(
    url="https://www.warcraftlogs.com/api/v2/client",
    headers={"Authorization": f"Bearer {access_token}"},
    use_json=True,
)

client = Client(transport=transport, fetch_schema_from_transport=False)

# Get report metadata for a given report code
report_code = "rVnDB4fcb9qZAPLv"  # change report code here
query = gql("""
query {
  reportData {
    report(code: "%s") {
      title
      startTime
      endTime
      zone {
        name
      }
      fights {
        id
        name
        startTime
        endTime
        kill
      }
    }
  }
}
""" % report_code)

result = client.execute(query)
print(result)
